## Stock Screener: Model Training and Evaluation

### Library Installation (if needed)

In [207]:
# Install libraries the first time
# !pip install -q ipynb yfinance pandas pathlib numpy

### Importing Required Libraries

In [208]:
from fastai.tabular.all import *
import yfinance as yf
import pandas as pd
import pathlib
from datetime import datetime, timedelta
import numpy as np
import matplotlib.pyplot as plt
from fastai.metrics import rmse, mae

### Global Variables

In [209]:
# Model settings
modelName = 'stockScreenerV6.0'
trainingData = 'stockData.csv'
getNewData = True 
trainNewModel = True
predictionTarget = 'NVO'  # 'ALL' for all tickers, 'None' for no prediction

# Training parameters
trainingSize = 150  # Number of stocks to use for training
timeFrame = 'max'   # Options: '1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y', 'ytd', 'max'
yNames = ['Future Year Change']
catNames = ['Date']
contNames = ['Open', 'Close', 'Volume', 'Dividends', 'Stock Splits', 'EV/EBIT', 'ROIC']
epochs = 3

# Testing parameters
testSize = 420  # Number of stocks to test, 'ALL' for all non-training stocks

# Paths
basePath = Path.cwd().parent
dataFolder = basePath / 'TrainingData'
modelFolder = basePath.parent / 'TrainedModels' / 'stockScreener'
testFolder = basePath / 'TestData'

## Data Preparation

#### Remove Duplicate Tickers

In [210]:
df = pd.read_csv(testFolder / 'tickers.csv')
df.drop_duplicates().to_csv(testFolder / 'tickers.csv', index=False)

#### Utility Functions

In [211]:
def getIndustry(tickerSymbol):
    """Fetches the industry of the given ticker symbol."""
    try:
        return yf.Ticker(tickerSymbol).info.get('industry', 'Unknown')
    except Exception as e:
        return 'Error'

In [212]:
def calculateFutureYearChange(ticker_symbol, timeframe, buffer=1):
    valid_periods = ['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y', 'ytd', 'max']
    try:
        if timeframe == 'max':
            future_change = yf.Ticker(ticker_symbol).history(period='max')
            future_change['Future Year Change'] = (future_change['Close'].shift(-252) / future_change['Close'] - 1)
            future_change = future_change.dropna(subset=['Future Year Change'])
        else:
            # Calculate the extended timeframe
            extended_timeframe = valid_periods[min(valid_periods.index(timeframe) + buffer, len(valid_periods) - 1)]
            future_change = yf.Ticker(ticker_symbol).history(period=extended_timeframe)
            future_change['Future Year Change'] = (future_change['Close'].shift(-252) / future_change['Close'] - 1)

            # Calculate the end date and start date based on the timeframe
            end_date = future_change.index[-1] - pd.DateOffset(years=1)
            start_date = end_date - pd.DateOffset(years=int(timeframe[:-1]))
            future_change = future_change.loc[start_date:end_date].dropna(subset=['Future Year Change'])
        return future_change
    except Exception as e:
        return pd.DataFrame()

In [213]:
def getEVComponents(tickerSymbol):
    """Fetches static EV components (total debt, cash, shares outstanding)."""
    try:
        info = yf.Ticker(tickerSymbol).info
        return info.get('totalDebt', 0), info.get('totalCash', 0), info.get('sharesOutstanding', None)
    except Exception as e:
        return None, None, None

In [214]:
def calculateApproxEBIT(tickerSymbol):
    """Estimates EBIT based on revenue and operating income."""
    try:
        info = yf.Ticker(tickerSymbol).info
        revenue = info.get('totalRevenue', None)
        operatingIncome = info.get('operatingIncome', None)
        return operatingIncome if operatingIncome else (revenue * 0.15 if revenue else None)
    except Exception as e:
        return None

In [215]:
def enrichDataWithMetrics(histData):
    """Adds EV/EBIT and ROIC to the historical data."""
    for ticker in histData['Ticker'].unique():
        try:
            totalDebt, cash, sharesOutstanding = getEVComponents(ticker)
            ebit = calculateApproxEBIT(ticker)

            if sharesOutstanding and ebit and ebit != 0:
                histData.loc[histData['Ticker'] == ticker, 'EV/EBIT'] = (
                    (histData['Close'] * sharesOutstanding + totalDebt - cash) / ebit
                )

                taxRate = 0.21
                nopat = ebit * (1 - taxRate)
                investedCapital = totalDebt + (histData['Close'] * sharesOutstanding) - cash
                histData.loc[histData['Ticker'] == ticker, 'ROIC'] = nopat / investedCapital
        except Exception as e:
            pass
    return histData

In [216]:
def getTickerDataFrom1YrAgo(ticker_symbol):
    try:
        # Fetch ticker data
        ticker = yf.Ticker(ticker_symbol)

        # Define the date range: one year ago to today
        today = datetime.today()
        one_year_ago = today - timedelta(days=365)

        # Fetch historical data for one year ago
        hist = ticker.history(start=(one_year_ago - timedelta(days=30)).strftime('%Y-%m-%d'), 
                              end=(one_year_ago + timedelta(days=0)).strftime('%Y-%m-%d'))
        if hist.empty:
            raise ValueError(f"No historical data available for {ticker_symbol} around {one_year_ago.strftime('%Y-%m-%d')}.")

        # Extract the closest data point to one year ago
        row = hist.iloc[0]  # Get the first available entry within the date range

        # Price today
        price_today = ticker.history(period='1d')['Close'].iloc[-1]

        # Calculate future price change (from one year ago to today)
        price_change_future = ((price_today - row['Close']) / row['Close']) if row['Close'] else None

        # Collect additional data
        total_debt, cash, shares_outstanding = getEVComponents(ticker_symbol)
        ebit = calculateApproxEBIT(ticker_symbol)
        ev = (row['Close'] * shares_outstanding) + total_debt - cash if shares_outstanding else None
        ev_ebit = ev / ebit if ebit else None
        market_cap = row['Close'] * shares_outstanding if shares_outstanding else None
        tax_rate = 0.21
        nopat = ebit * (1 - tax_rate) if ebit else None
        invested_capital = total_debt + market_cap - cash if market_cap and total_debt and cash else None
        roic = nopat / invested_capital if nopat and invested_capital else None
        industry = getIndustry(ticker_symbol)

        # Return as a DataFrame
        return pd.DataFrame([{
            'Date': row.name,
            'Open': row['Open'],
            'High': row['High'],
            'Low': row['Low'],
            'Close': row['Close'],
            'Volume': row['Volume'],
            'Dividends': row.get('Dividends', 0.0),
            'Stock Splits': row.get('Stock Splits', 0.0),
            'Future Year Change': price_change_future,
            'Industry': industry,
            'EV/EBIT': ev_ebit,
            'ROIC': roic
        }])

    except Exception as e:
        print(f"Error fetching data for {ticker_symbol}: {e}")
        return pd.DataFrame()

In [217]:
def getTickerData(ticker_symbol):
    try:
        ticker = yf.Ticker(ticker_symbol)
        hist = ticker.history(period='1d')
        total_debt, cash, shares_outstanding = getEVComponents(ticker_symbol)
        ebit = calculateApproxEBIT(ticker_symbol)
        ev = (hist['Close'].iloc[-1] * shares_outstanding) + total_debt - cash
        ev_ebit = ev / ebit if ebit else None
        market_cap = hist['Close'].iloc[-1] * shares_outstanding
        tax_rate = 0.21
        nopat = ebit * (1 - tax_rate) if ebit else None
        invested_capital = total_debt + market_cap - cash
        roic = nopat / invested_capital if nopat and invested_capital else None
        industry = getIndustry(ticker_symbol)
        
        # Add the 'Date' column
        date = datetime.now().strftime('%Y-%m-%d')

        return pd.DataFrame([{
            'Date': date,
            'Open': hist['Open'].iloc[-1],
            'High': hist['High'].iloc[-1],
            'Low': hist['Low'].iloc[-1],
            'Close': hist['Close'].iloc[-1],
            'Volume': hist['Volume'].iloc[-1],
            'Dividends': hist.get('Dividends', pd.Series([0.0])).iloc[-1],
            'Stock Splits': hist.get('Stock Splits', pd.Series([0.0])).iloc[-1],
            'EV/EBIT': ev_ebit,
            'Market Cap': market_cap,
            'ROIC': roic,
            'Industry': industry
        }])
    except Exception as e:

        return pd.DataFrame()

#### Download and Process Historical Data

In [218]:
tickers = pd.read_csv(testFolder / 'tickers.csv')['Ticker']
trainingTickers = np.random.choice(tickers, size=trainingSize, replace=False)

if getNewData:
    histData = pd.DataFrame()

    for ticker in trainingTickers:
        try:
            data = calculateFutureYearChange(ticker, timeFrame)
            data['Ticker'] = ticker
            data['Industry'] = getIndustry(ticker)
            histData = pd.concat([histData, data])
        except Exception:
            pass

    histData.reset_index(inplace=True)
    histData = enrichDataWithMetrics(histData)
    histData.to_csv(dataFolder / trainingData, index=False)
    print(f"Training data saved: {len(histData)} rows")

$PVG: possibly delisted; no timezone found
C:\Users\Gamer\AppData\Local\Temp\ipykernel_6480\223849276.py:12: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  histData = pd.concat([histData, data])
$GUT-A: possibly delisted; no timezone found
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/GUT-A?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=GUT-A&crumb=TLJxoxQPcjU
C:\Users\Gamer\AppData\Local\Temp\ipykernel_6480\223849276.py:12: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, e

Training data saved: 506704 rows


### Model Training

In [219]:
# Remove rows with no EV/EBIT- or ROIC data
df = pd.read_csv(dataFolder / trainingData)
dfCleaned = df.dropna(subset=['EV/EBIT', 'ROIC'])
dfCleaned.to_csv(dataFolder / trainingData, index=False)

splits = RandomSplitter(valid_pct=0.2)(range_of(dfCleaned))

to = TabularPandas(
    df, procs=[Categorify, FillMissing, Normalize],
    y_names=yNames, cat_names=catNames, cont_names=contNames, splits=splits)

dls = to.dataloaders(bs=64)

c:\Users\Gamer\miniconda3\Lib\site-packages\fastai\tabular\core.py:314: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  to[n].fillna(self.na_dict[n], inplace=True)
c:\Users\Gamer\miniconda3\Lib\site-packages\fastai\tabular\core.py:314: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

Fo

In [220]:
if trainNewModel:
    learn = tabular_learner(dls, metrics=[rmse, mae])
    learn.fit_one_cycle(epochs)
    learn.export(modelFolder / f'{modelName}.pkl')
    print("Model training complete")

epoch,train_loss,valid_loss,_rmse,mae,time


Model training complete.


## Testing and benchmarking

In [221]:
def logEvaluation(model_name, mae, rmse, r2, model_folder, test_tickers):
    """Log evaluation metrics to CSV file"""
    log_file = model_folder / "modelEvaluations.csv"
    
    new_entry_df = pd.DataFrame([{
        "Model Name": modelName,
        "Timestamp": datetime.now().strftime('%Y-%m-%d %H:%M'),
        "MAE": f'{mae:.3f}',
        "RMSE": f'{rmse:.3f}',
        "R2": f'{r2:.3f}',
        "Epochs": epochs,
        "Training Size": trainingSize,
        "Test Size": len(test_tickers),
        "Cat Names": catNames,
        "Cont Names": contNames,
    }])
    
    try:
        log_df = pd.read_csv(log_file)
        log_df = pd.concat([log_df, new_entry_df], ignore_index=True)
    except FileNotFoundError:
        log_df = new_entry_df
        
    log_df.to_csv(log_file, index=False)
    print(f"Logged evaluation results to {log_file}")

In [222]:
def plotResults(filtered_df, model_name, model_folder):
    """Create and save visualization plots"""
    plt.figure(figsize=(12, 8))
    
    # Scatter plot
    plt.subplot(2, 1, 1)
    actuals = filtered_df['Actual']
    predictions = filtered_df['Predicted']
    plt.scatter(actuals, predictions, alpha=0.7, label='Predictions')
    
    # Perfect prediction line
    min_val = min(actuals.min(), predictions.min())
    max_val = max(actuals.max(), predictions.max())
    plt.plot([min_val, max_val], [min_val, max_val], 
             color='red', linestyle='--', label='Perfect Prediction')
    
    plt.title(f'Predicted vs. Actual Returns - {model_name}', fontsize=14)
    plt.xlabel('Actual Returns', fontsize=12)
    plt.ylabel('Predicted Returns', fontsize=12)
    plt.legend()
    plt.grid(alpha=0.5)
    
    # Residual plot
    plt.subplot(2, 1, 2)
    plt.scatter(predictions, filtered_df['Residual'], alpha=0.7)
    plt.axhline(y=0, color='r', linestyle='--')
    plt.title('Residual Plot', fontsize=14)
    plt.xlabel('Predicted Returns', fontsize=12)
    plt.ylabel('Residual', fontsize=12)
    plt.grid(alpha=0.5)
    
    plt.tight_layout()
    
    # Show plot
    plt.show()

In [233]:
def evaluateModel(learn, test_tickers, model_name, model_folder, cont_names, cat_names):
    all_predictions = []
    all_actuals = []
    test_data_list = []

    for ticker in test_tickers:
        try:
            test_data = getTickerDataFrom1YrAgo(ticker)
            if test_data.empty:
                print(f"Skipping {ticker} due to insufficient valid data")
                continue
            test_data_list.append(test_data)
        except Exception:
            continue

    if not test_data_list:
        print("No valid test data collected")
        return None, None, None

    combined_test_data = pd.concat(test_data_list, ignore_index=True)
    test_dl = learn.dls.test_dl(combined_test_data)
    preds, targs = learn.get_preds(dl=test_dl)
    predictions = preds.numpy()
    actuals = targs.numpy()

    results_df = pd.DataFrame({
        'Predicted': predictions.flatten(),
        'Actual': actuals.flatten()
    })

    results_df['Residual'] = results_df['Actual'] - results_df['Predicted']
    outlier_threshold = 2 * results_df['Residual'].std()
    filtered_df = results_df[abs(results_df['Residual']) <= outlier_threshold]

    mae = np.mean(np.abs(filtered_df['Residual']))
    rmse = np.sqrt(np.mean(filtered_df['Residual']**2))
    r2 = 1 - (np.sum(filtered_df['Residual']**2) / np.sum((filtered_df['Actual'] - filtered_df['Actual'].mean())**2))

    logEvaluation(model_name, mae, rmse, r2, model_folder, test_tickers)
    plotResults(filtered_df, model_name, model_folder)

    return mae, rmse, r2

In [234]:
test = getTickerDataFrom1YrAgo("NVO")
test.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Future Year Change,Industry,EV/EBIT,ROIC
0,2023-12-26 00:00:00-05:00,101.445632,102.009439,101.297268,101.564331,1851400.0,0.0,0.0,-0.202181,Drug Manufacturers - General,7.979916,0.098999


## Evaluate model

In [235]:
if trainNewModel:
    nonTrainingTickers = list(set(tickers) - set(trainingTickers))
    randomNonTrainingTickers = np.random.choice(nonTrainingTickers, size=len(nonTrainingTickers), replace=False).tolist()
    
#randomNonTrainingTickers = np.random.choice(nonTrainingTickers, size=min(testSize, len(nonTrainingTickers)), replace=False).tolist()

    # Filter out test tickers with NaN EV/EBIT or NaN ROIC
    validTestTickers = []
    i = 0
    while i < testSize:
        testData = getTickerDataFrom1YrAgo(randomNonTrainingTickers[i])
        if testData.empty or testData[['EV/EBIT', 'ROIC']].isna().any().any():
            pass
        else:
            validTestTickers.append(testData)
        i += 1

    if not validTestTickers:
        raise ValueError("No valid test data collected. Ensure testTickers have valid EV/EBIT and ROIC data.")

    combinedTestData = pd.concat(validTestTickers, ignore_index=True)

    mae, rmse, r2 = evaluateModel(
        learn=learn,
        test_tickers=combinedTestData,
        model_name=modelName,
        model_folder=modelFolder,
        cont_names=contNames,
        cat_names=catNames
    )
    
    print(f"Amount of test tickers: {len(combinedTestData)}, Expected: {testSize}")

    if mae is not None and rmse is not None and r2 is not None:
        print(f"Evaluation Results:")
        print(f"MAE: {mae:.3f}")
        print(f"RMSE: {rmse:.3f}")
        print(f"R2: {r2:.3f}")
    else:
        print("Evaluation failed. Metrics are None.")

$BSI: possibly delisted; no price data found  (1d 2023-12-24 -> 2024-01-23)


Error fetching data for BSI: No historical data available for BSI around 2024-01-23.


$CMS-B: possibly delisted; no timezone found
$GEQ: possibly delisted; no price data found  (1d 2023-12-24 -> 2024-01-23)


Error fetching data for CMS-B: No historical data available for CMS-B around 2024-01-23.
Error fetching data for GEQ: No historical data available for GEQ around 2024-01-23.


$MTU: possibly delisted; no price data found  (1d 2023-12-24 -> 2024-01-23)


...

Error fetching data for Industry: No historical data available for Industry around 2024-01-23.
Skipping Industry due to insufficient valid data


Failed to get ticker 'EV/EBIT' reason: Expecting value: line 1 column 1 (char 0)
$EV/EBIT: possibly delisted; no timezone found


Error fetching data for EV/EBIT: No historical data available for EV/EBIT around 2024-01-23.
Skipping EV/EBIT due to insufficient valid data


C:\Users\Gamer\AppData\Local\Temp\ipykernel_6480\1893558532.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_test_data = pd.concat(test_data_list, ignore_index=True)


AssertionError: nan values in `EV/EBIT` but not in setup training set

## Tests

In [226]:
print('Model files in modelFolder:')
for file in modelFolder.glob('*.pkl'):
    print(file.name)

Model files in modelFolder:
stockScreenerV1.0.pkl
stockScreenerV2.0.pkl
stockScreenerV3.0.pkl
stockScreenerV4.0.pkl
stockScreenerV4.1.pkl
stockScreenerV5.0.pkl
stockScreenerV5.1.pkl
stockScreenerV5.2.pkl
stockScreenerV5.3.pkl
stockScreenerV5.4.pkl
stockScreenerV6.0.pkl


In [227]:
evaluations = pd.read_csv(modelFolder / 'modelEvaluations.csv')
bestModel = evaluations.sort_values('MAE', ascending=True).iloc[0]
bestModel.head()

Model Name    stockScreenerV5.0
Timestamp      2025-01-21 21:03
MAE                       0.237
RMSE                      0.302
R2                       -0.002
Name: 5, dtype: object

### Load model

In [228]:
if os.name == 'nt':
    temp = pathlib.PosixPath
    pathlib.PosixPath = pathlib.WindowsPath

importedModel = Path(f"{bestModel['Model Name']}.pkl") # Change this if you want to try other models
learn = load_learner(modelFolder / importedModel)

## Predictions

In [229]:
if predictionTarget != 'None':
    if predictionTarget == 'ALL':
        predictionTickers = tickers
    elif predictionTarget.endswith('%'):
        percentage = float(predictionTarget.strip('%')) / 100
        num_tickers = int(len(tickers) * percentage)
        predictionTickers = np.random.choice(tickers, size=num_tickers, replace=False).tolist()
    else:
        predictionTickers = [predictionTarget]

    # Fetch data for prediction tickers
    dfPrediction = pd.concat([getTickerData(ticker) for ticker in predictionTickers], ignore_index=True)

    # Ensure dfPrediction is a DataFrame
    if isinstance(dfPrediction, dict):
        dfPrediction = pd.DataFrame([dfPrediction])

    # Create test dataloader
    dl = learn.dls.test_dl(dfPrediction)
    dfPrediction.head()

    # Make predictions
    prediction = learn.get_preds(dl=dl)
    adr_df = pd.read_csv(testFolder / 'tickers.csv')
    company_dict = dict(zip(adr_df['Ticker'], adr_df['Company']))

    if predictionTarget == 'ALL' or predictionTarget.endswith('%'):
        sorted_predictions = sorted(zip(predictionTickers, prediction[0]), key=lambda x: x[1], reverse=True)
        print(f"Got predictions for {len(sorted_predictions)} tickers, expected: {len(predictionTickers)}")
        print(f"Prediction for best performing tickers:")
        for symbol, pred in sorted_predictions:
            company_name = company_dict.get(symbol, 'Unknown')
            print(f"{symbol} ({company_name}): {pred[0].item() * 100:.2f}%")
    else:
        company_name = company_dict.get(predictionTarget, 'Unknown')
        print(f"Prediction for {predictionTarget} ({company_name}):")
        print(f"{prediction[0][0][0].item() * 100:.2f}%")
    print("Free money?!")

predictionTarget = '8%'

Prediction for NVO (Novo Nordisk A/S):
41.37%
Free money?!
